# Zero to Snowflake - Governance with Horizon

**Asset:** Zero to Snowflake - Governance with Horizon  
**Version:** v1  
**Copyright(c):** 2025 Snowflake Inc. All rights reserved.

## Horizonによるガバナンス
1. ロールとアクセス制御の紹介
2. 自動タグ付けによるタグベース分類
3. マスキングポリシーによる列レベルセキュリティ
4. 行アクセスポリシーによる行レベルセキュリティ
5. データメトリック関数によるデータ品質監視
6. Trust Centerによるアカウントセキュリティ監視

In [ ]:
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "governance_with_horizon"}}';

まず、ワークシートコンテキストを設定しましょう

In [ ]:
USE ROLE useradmin;
USE DATABASE tb_101;
USE WAREHOUSE tb_dev_wh;

## 1. ロールとアクセス制御の紹介

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/security-access-control-overview

Snowflakeアクセス制御フレームワークは以下に基づいています：
- **ロールベースアクセス制御（RBAC）:** アクセス権限がロールに割り当てられ、ロールがユーザーに割り当てられます。
- **任意アクセス制御（DAC）:** 各オブジェクトには所有者がおり、所有者がそのオブジェクトへのアクセスを許可できます。

### 主要概念:
- **セキュリティ保護可能オブジェクト:** 誰が使用または表示できるかを制御できるもの
- **ロール:** 配布可能な権限のセット
- **権限:** オブジェクトに対して何かを行う特定の許可
- **ユーザー:** Snowflakeが認識するアイデンティティ

### Snowflakeシステム定義ロール:
- **ORGADMIN:** 組織レベルでの操作を管理
- **ACCOUNTADMIN:** システムの最上位ロール
- **SECURITYADMIN:** グローバルにオブジェクトグラントを管理
- **USERADMIN:** ユーザーとロール管理専用
- **SYSADMIN:** ウェアハウスとデータベースを作成する権限を持つ
- **PUBLIC:** すべてのユーザーとロールに自動的に付与される疑似ロール

In [ ]:
SHOW ROLES;

カスタムデータスチュワードロールを作成します

In [ ]:
CREATE OR REPLACE ROLE tb_data_steward
    COMMENT = 'カスタムロール';

SECURITYADMINロールに切り替えて新しいロールに権限を付与

In [ ]:
USE ROLE securityadmin;

In [ ]:
GRANT OPERATE, USAGE ON WAREHOUSE tb_dev_wh TO ROLE tb_data_steward;

In [ ]:
GRANT USAGE ON DATABASE tb_101 TO ROLE tb_data_steward;
GRANT USAGE ON ALL SCHEMAS IN DATABASE tb_101 TO ROLE tb_data_steward;

In [ ]:
GRANT SELECT ON ALL TABLES IN SCHEMA raw_customer TO ROLE tb_data_steward;

In [ ]:
GRANT ALL ON SCHEMA governance TO ROLE tb_data_steward;
GRANT ALL ON ALL TABLES IN SCHEMA governance TO ROLE tb_data_steward;

現在のユーザーに新しいロールを付与

In [ ]:
SET my_user = CURRENT_USER();
GRANT ROLE tb_data_steward TO USER IDENTIFIER($my_user);

In [ ]:
USE ROLE tb_data_steward;

In [ ]:
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

## 2. 自動タグ付けによるタグベース分類

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/classify-auto

Snowflakeの自動タグ付け機能をタグベースマスキングと組み合わせて使用し、クエリ結果の機密データを難読化できます。

In [ ]:
USE ROLE accountadmin;

In [ ]:
CREATE OR REPLACE TAG governance.pii;
GRANT APPLY TAG ON ACCOUNT TO ROLE tb_data_steward;

In [ ]:
GRANT EXECUTE AUTO CLASSIFICATION ON SCHEMA raw_customer TO ROLE tb_data_steward;
GRANT DATABASE ROLE SNOWFLAKE.CLASSIFICATION_ADMIN TO ROLE tb_data_steward;
GRANT CREATE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE ON SCHEMA governance TO ROLE tb_data_steward;

In [ ]:
USE ROLE tb_data_steward;

分類プロファイルを作成

In [ ]:
CREATE OR REPLACE SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE
  governance.tb_classification_profile(
    {
      'minimum_object_age_for_classification_days': 0,
      'maximum_classification_validity_days': 30,
      'auto_tag': true
    });

タグマップを作成して自動タグ付けを設定

In [ ]:
CALL governance.tb_classification_profile!SET_TAG_MAP(
  {'column_tag_map':[
    {
      'tag_name':'tb_101.governance.pii',
      'tag_value':'pii',
      'semantic_categories':['NAME', 'PHONE_NUMBER', 'POSTAL_CODE', 'DATE_OF_BIRTH', 'CITY', 'EMAIL']
    }]});

テーブルを分類

In [ ]:
CALL SYSTEM$CLASSIFY('tb_101.raw_customer.customer_loyalty', 'tb_101.governance.tb_classification_profile');

自動分類とタグ付けの結果を確認

In [ ]:
SELECT 
    column_name,
    tag_database,
    tag_schema,
    tag_name,
    tag_value,
    apply_method
FROM TABLE(INFORMATION_SCHEMA.TAG_REFERENCES_ALL_COLUMNS('raw_customer.customer_loyalty', 'table'));

## 3. マスキングポリシーによる列レベルセキュリティ

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/security-column-intro

Snowflakeの列レベルセキュリティでは、マスキングポリシーを使用して列内のデータを保護できます。

機密文字列データ用のマスキングポリシーを作成

In [ ]:
CREATE OR REPLACE MASKING POLICY governance.mask_string_pii AS (original_value STRING)
RETURNS STRING ->
  CASE WHEN
    CURRENT_ROLE() NOT IN ('ACCOUNTADMIN', 'TB_ADMIN')
    THEN '****MASKED****'
    ELSE original_value
  END;

機密DATEデータ用のマスキングポリシーを作成

In [ ]:
CREATE OR REPLACE MASKING POLICY governance.mask_date_pii AS (original_value DATE)
RETURNS DATE ->
  CASE WHEN
    CURRENT_ROLE() NOT IN ('ACCOUNTADMIN', 'TB_ADMIN')
    THEN DATE_TRUNC('year', original_value)
    ELSE original_value
  END;

マスキングポリシーをタグに添付

In [ ]:
ALTER TAG governance.pii SET
    MASKING POLICY governance.mask_string_pii,
    MASKING POLICY governance.mask_date_pii;

publicロールでマスキングをテスト

In [ ]:
USE ROLE public;
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

管理者ロールではマスキングされないことを確認

In [ ]:
USE ROLE tb_admin;
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

## 4. 行アクセスポリシーによる行レベルセキュリティ

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/security-row-intro

Snowflakeは、行アクセスポリシーを使用した行レベルセキュリティをサポートして、クエリ結果で返される行を決定します。

In [ ]:
USE ROLE tb_data_steward;

行ポリシーマップを作成

In [ ]:
CREATE OR REPLACE TABLE governance.row_policy_map
    (role STRING, country_permission STRING);

In [ ]:
INSERT INTO governance.row_policy_map
    VALUES('tb_data_engineer', 'United States');

行アクセスポリシーを作成

In [ ]:
CREATE OR REPLACE ROW ACCESS POLICY governance.customer_loyalty_policy
    AS (country STRING) RETURNS BOOLEAN ->
        CURRENT_ROLE() IN ('ACCOUNTADMIN', 'SYSADMIN') 
        OR EXISTS 
            (
            SELECT 1
                FROM governance.row_policy_map rp
            WHERE
                UPPER(rp.role) = CURRENT_ROLE()
                AND rp.country_permission = country
            );

テーブルにポリシーを適用

In [ ]:
ALTER TABLE raw_customer.customer_loyalty
    ADD ROW ACCESS POLICY governance.customer_loyalty_policy ON (country);

行ポリシーをテスト

In [ ]:
USE ROLE tb_data_engineer;

In [ ]:
SELECT TOP 100 * FROM raw_customer.customer_loyalty;

## 5. データメトリック関数によるデータ品質監視

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/data-quality-intro

Snowflakeは、データメトリック関数（DMF）を使用してデータの一貫性と信頼性を維持します。

In [ ]:
USE ROLE tb_data_steward;

システムDMFの例

In [ ]:
SELECT SNOWFLAKE.CORE.NULL_PERCENT(SELECT customer_id FROM raw_pos.order_header);

In [ ]:
SELECT SNOWFLAKE.CORE.DUPLICATE_COUNT(SELECT order_id FROM raw_pos.order_header);

In [ ]:
SELECT SNOWFLAKE.CORE.AVG(SELECT order_total FROM raw_pos.order_header);

カスタムデータメトリック関数を作成

In [ ]:
CREATE OR REPLACE DATA METRIC FUNCTION governance.invalid_order_total_count(
    order_prices_t table(
        order_total NUMBER,
        unit_price NUMBER,
        quantity INTEGER
    )
)
RETURNS NUMBER
AS
'SELECT COUNT(*)
 FROM order_prices_t
 WHERE order_total != unit_price * quantity';

テスト用の不正確なデータを挿入

In [ ]:
INSERT INTO raw_pos.order_detail
SELECT
    904745311,
    459520442,
    52,
    null,
    0,
    2,
    5.0,
    5.0,
    null;

カスタムDMFを実行

In [ ]:
SELECT governance.invalid_order_total_count(
    SELECT 
        price, 
        unit_price, 
        quantity 
    FROM raw_pos.order_detail
) AS num_orders_with_incorrect_price;

テーブルにDMFを設定

In [ ]:
ALTER TABLE raw_pos.order_detail
    SET DATA_METRIC_SCHEDULE = 'TRIGGER_ON_CHANGES';

In [ ]:
ALTER TABLE raw_pos.order_detail
    ADD DATA METRIC FUNCTION governance.invalid_order_total_count
    ON (price, unit_price, quantity);

## 6. Trust Centerによるアカウントセキュリティ監視

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/trust-center/overview

Trust Centerは、スキャナーを使用してアカウントのセキュリティリスクを評価・監視する自動チェックを可能にします。

### 一般的なユースケース:
- ユーザーの多要素認証が有効になっていることを確認
- 過剰に特権化されたロールの発見
- 最低90日間ログインしていない非アクティブユーザーの発見
- リスクの高いユーザーの発見と軽減

Trust Centerは、ナビゲーションメニューの「監視」ボタンから「Trust Center」をクリックしてアクセスできます。

In [ ]:
USE ROLE accountadmin;
GRANT APPLICATION ROLE SNOWFLAKE.TRUST_CENTER_ADMIN TO ROLE tb_admin;
USE ROLE tb_admin;

---
## RESET
---

In [ ]:
USE ROLE accountadmin;

In [ ]:
DROP ROLE IF EXISTS tb_data_steward;

In [ ]:
ALTER TAG IF EXISTS governance.pii UNSET
    MASKING POLICY governance.mask_string_pii,
    MASKING POLICY governance.mask_date_pii;
DROP MASKING POLICY IF EXISTS governance.mask_string_pii;
DROP MASKING POLICY IF EXISTS governance.mask_date_pii;

In [ ]:
ALTER SCHEMA raw_customer UNSET CLASSIFICATION_PROFILE;
DROP SNOWFLAKE.DATA_PRIVACY.CLASSIFICATION_PROFILE IF EXISTS tb_classification_profile;

In [ ]:
ALTER TABLE raw_customer.customer_loyalty 
    DROP ROW ACCESS POLICY governance.customer_loyalty_policy;
DROP ROW ACCESS POLICY IF EXISTS governance.customer_loyalty_policy;

In [ ]:
DELETE FROM raw_pos.order_detail WHERE order_detail_id = 904745311;
ALTER TABLE raw_pos.order_detail
    DROP DATA METRIC FUNCTION governance.invalid_order_total_count ON (price, unit_price, quantity);
DROP FUNCTION governance.invalid_order_total_count(TABLE(NUMBER, NUMBER, INTEGER));
ALTER TABLE raw_pos.order_detail UNSET DATA_METRIC_SCHEDULE;

In [ ]:
DROP TAG IF EXISTS governance.pii;

In [ ]:
ALTER SESSION UNSET query_tag;

In [ ]:
ALTER WAREHOUSE tb_dev_wh SUSPEND;